In [2]:
import os
import ast
import openai
import time
import json
from json import JSONDecodeError
import re

In [3]:
# pip install openai==0.28

In [4]:
# openai.organization = ""  #My OpenAI account
# openai.api_key = "" #My Key
# openai.Model.list()

<OpenAIObject list at 0x1dce7c4eea0> JSON: {
  "object": "list",
  "data": [
    {
      "id": "dall-e-3",
      "object": "model",
      "created": 1698785189,
      "owned_by": "system"
    },
    {
      "id": "dall-e-2",
      "object": "model",
      "created": 1698798177,
      "owned_by": "system"
    },
    {
      "id": "gpt-3.5-turbo-0125",
      "object": "model",
      "created": 1706048358,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-ada-002",
      "object": "model",
      "created": 1671217299,
      "owned_by": "openai-internal"
    },
    {
      "id": "tts-1-hd-1106",
      "object": "model",
      "created": 1699053533,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-3-small",
      "object": "model",
      "created": 1705948997,
      "owned_by": "system"
    },
    {
      "id": "gpt-4-0613",
      "object": "model",
      "created": 1686588896,
      "owned_by": "openai"
    },
    {
      "id": "tts-1-hd",
      "object

In [5]:
with open('llm_summarization_by_gpt4.json', 'r', encoding='utf-8') as summary:
    summary_data = json.load(summary)
print(len(summary_data))

1200


In [6]:
# summary_data[0]

In [7]:
with open('evidence_corpus.json', 'r', encoding='utf-8') as f:
    evidence_data = json.load(f)
print(len(evidence_data))

8096


In [8]:
# Call GPT3.5 Model 
def get_completion_from_messages(messages, 
                                 model= "gpt-4-1106-preview", #gpt-3.5-turbo-16k-0613" 
                                 temperature=0, 
                                 max_tokens=1024):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
        response_format={"type": "json_object"}
    )
    return response.choices[0].message["content"]

In [8]:
delimiter_1 = "#"
delimiter_2 = "##"

system_message = f"""
You are a Fact Checker.\
The user will give you a news paragraph and several evidence abstracts.\
The evidence abstracts may contain information that upholds or contradicts the news article’s central claim. \
Determine whether the news story is factually true or fake based on the evidence abstracts.\ 
The user will put the news paragraph behind {delimiter_1} and the relevant abstracts behind {delimiter_2}. 
"""


In [9]:
generated_result = []
# i = 321, 112, 333
i = 0

In [34]:
for item in summary_data[975:]:
    print(f"\n\nPredicting {i} :-")
    
    news = item['abstractive']
#     print(news)
    
    evidence_list = []
    if type(item['evidence'])==str:
        evidence_list.append(item['evidence'])
    else:
        evidence_list.extend(item['evidence'])
    print(f"evidence list: {evidence_list}")
    
    evidence = [it['abstract'] for it in evidence_data if it['cord_uid'] in evidence_list]
    l = len(evidence_list)
    evidence = ''.join(evidence)
#     print(evidence)
    
    user_message = f"""
    Here is one news paragraph behind {delimiter_1}{news} and {l} relevant abstracts behind {delimiter_2}{evidence}.\
    Determine if these {l} separate statements support or refute the point of view in the news paragraph.\
    Finally provide an answer of 'True' if the news paragraph is factually true or\
    'False' if it is false based on the evidence with reasons.\
    Provide the final output in JSON format with the keys: prediction and reason.
    """
    
#     user_message = f"""
#     Here is one news paragraph behind {delimiter_1}{news} and {l} relevant abstracts behind {delimiter_2}{evidence}.\
#     Determine in {l} separate statements if these abstracts uphold or contradict the point of view in the news paragraph.\
#     Finally provide a conclusion if the news paragraph is factually True or False based on the evidence.
#     """
#     messages =  [  
#                 {'role':'system', 'content': system_message},    
#                 {'role':'user', 'content': user_message},  
#                 ]
    
    assistant_message = """
    As a Fact Checker, carefully analyze the evidence abstracts and provide the result in JSON format. \
    Reference specific phrases or sentences from both the news story and the evidence to support your prediction.
    """

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message},
        {'role': 'assistant', 'content': assistant_message},
    ]
    
    response = get_completion_from_messages(messages)
    print(response)
    
    result = dict()
    result['news'] = news
    result['evidence list'] = evidence_list
    result['ground truth'] = item['label']
    
    try:
        result.update(json.loads(response))
        
    except JSONDecodeError as e:
        pattern = r'"prediction": "(.*?)",\s*"reason": "(.*?)"'
        match = re.search(pattern, response)
        if match:
            result['prediction'] = match.group(1)
            result['reason'] = match.group(2)
#             print(dict({'prediction':result['prediction'], 'reason':result['reason']})) 
    
    generated_result.append(result)
    
    time.sleep(5)
    i += 1
    
#     break



Predicting 975 :-
evidence list: ['dicwp1dm', 'dicwp1dm', 'rt2i2wca']
{
  "prediction": "True",
  "reason": "The news paragraph's central claim is that there has been an increase in the emergence and resurgence of infectious diseases globally, with a disproportionate impact on resource-poor nations. The first evidence abstract supports this claim by discussing the significant epidemic threats posed by emerging infectious diseases like Ebola Virus Disease (EVD), Nipah Virus Encephalitis, and Lassa fever, particularly in resource-constrained regions. It also mentions the ethical challenges and the need for an ethical framework in response to such outbreaks, which aligns with the news paragraph's mention of the need for immediate action to mitigate contributing factors. The second evidence abstract is a duplicate of the first one and therefore supports the claim for the same reasons. The third evidence abstract directly supports the news paragraph by detailing the impact of emerging inf



Predicting 982 :-
evidence list: ['sk9lud0o', '5oerh31q', 'sk9lud0o']
{
  "prediction": "True",
  "reason": "The first and third evidence abstracts directly support the news paragraph's claim about a new method for calculating the basic reproduction number (R(0)) for infectious diseases, specifically HIV, without relying on the assumption of exponential case growth. Both abstracts mention that the study provides a simple framework for estimating R(0) and that the method has been applied to the HIV epidemic in European countries, yielding R(0) values ranging from 3.60 to 3.74. These values are consistent with those based on the Euler-Lotka equation, which aligns with the news paragraph's claim. The second evidence abstract discusses a different disease (COVID-19) and is not directly relevant to the claim about HIV in the news paragraph. However, it does not contradict the claim about the new method for calculating R(0) for HIV. Therefore, based on the evidence provided, the news parag



Predicting 991 :-
evidence list: ['tilwqk15', '3dt5uma6', 'tilwqk15']
{
  "prediction": "True",
  "reason": "The news paragraph's central claim is that researchers have developed a novel approach to combat advanced prostate cancer by targeting the interaction between the chaperone protein Hsp27 and the initiation factor eIF4E. The first evidence abstract supports this claim by detailing the identification of the interaction site between Hsp27 and eIF4E, the increased chemo- and hormone-sensitivity upon disrupting this interaction, and the identification of a phenazine derivative that can interfere with the Hsp27/eIF4E interaction. The second evidence abstract discusses a different approach using silibinin derivatives to target AR-positive prostate cancer cells, which does not directly support or refute the news paragraph's claim about Hsp27/eIF4E interaction but is relevant to the broader context of advanced prostate cancer treatment. The third evidence abstract is identical to the f



Predicting 999 :-
evidence list: ['h1eb764v', 'zf81s647', 'twxat0d6']
{
  "prediction": "True",
  "reason": "The third evidence abstract directly supports the news paragraph's claim about the ongoing HIV epidemic among MSM in the Netherlands. It discusses the same ATHENA cohort and identifies 106 large transmission clusters, with 91 being predominantly MSM and at least 54 existing before the introduction of cART in 1996. It also mentions that these clusters have persisted to the present day and that the reproduction numbers are around the critical threshold, indicating sustained transmission. The evidence abstract also notes the increase in the mean age of MSM diagnosed within these clusters and the emergence of new clusters with younger mean ages, which is consistent with the news paragraph's findings. The first and second evidence abstracts do not directly support or refute the news paragraph as they discuss different locations (Philippines and China) and do not provide information



Predicting 1007 :-
evidence list: ['ugpc7fow', '7uqmpn70', 'ugpc7fow']
{
  "prediction": "True",
  "reason": "Both evidence abstracts provided support the claims made in the news paragraph. The first and third abstracts are identical and confirm the historical use of Fomitopsis betulina in natural medicine, its antimicrobial, anticancer, and anti-inflammatory properties, and the pharmacological research that supports its various biological activities. These activities include antibacterial, anti-parasitic, antiviral, anti-inflammatory, anticancer, neuroprotective, and immunomodulating properties. They also mention the presence of triterpenoids and valuable enzymes, which aligns with the news paragraph's mention of these substances being used in healthcare product development and biotechnological applications. The second abstract does not directly relate to Fomitopsis betulina but discusses the potential of using a different fungus, Hypocrea jecorina (Trichoderma reesei), for biotechn

...



Predicting 1162 :-
evidence list: ['oauvgvsc', '404ocj5y', 'h600v8cn']
{
  "prediction": "True",
  "reason": "The news paragraph claims that recent observational research indicates tocilizumab may enhance survival and clinical outcomes for hospitalized COVID-19 patients. The first evidence abstract supports this by showing that tocilizumab treatment is associated with the reduction of biomarkers of COVID-19 infection. The second evidence abstract does not directly refute or support the survival benefit but indicates that tocilizumab use was not associated with increased microbial isolation, emergence of resistant organisms, or mortality benefits, but it does not contradict the potential for improved clinical outcomes in other aspects. The third evidence abstract directly supports the news paragraph by reporting that tocilizumab-treated patients showed improved oxygenation and inflammatory biomarkers, with higher than expected survival. All three abstracts together support the claim m



Predicting 1170 :-
evidence list: ['kjxnanke', '9aj7n0b5', 'kjxnanke']
{
  "prediction": "False",
  "reason": "The news paragraph itself states that 'the article is designed with the intention of containing false statements and serves as a teaching tool for students to practice critical analysis of content.' This admission indicates that the news story is not intended to be factually true. Despite the evidence abstracts supporting the content of the study and its findings, the news paragraph's own disclaimer about containing false statements for educational purposes overrides the factual alignment between the news content and the evidence provided."
}


Predicting 1171 :-
evidence list: ['kgl3r43i', '95x4lxla', 'ym3bp5n0']
{
  "prediction": "True",
  "reason": "The news paragraph's central claim is that a study from Wenzhou City, Zhejiang, China, has raised concerns about the adequacy of existing discharge criteria for COVID-19 patients due to instances of recurrent positive nucleic 



Predicting 1179 :-
evidence list: ['vhx6n727', '06c39gn7', 'ytxqy6px']
{
  "prediction": "True",
  "reason": "The news paragraph claims that a study at the Hospital Clinico San Carlos in Madrid found that SARS-CoV-2 RNA was detectable in the conjunctival swabs of patients with COVID-19, regardless of the presence of conjunctivitis, which is supported by the evidence abstracts. The first and second abstracts are identical and confirm that the study was conducted at the Hospital Clinico San Carlos in Madrid, included 36 subjects, and found SARS-CoV-2 RNA in the conjunctival swabs of two patients, one with conjunctivitis and one without, which matches the news paragraph's claim. The third abstract provides additional support by describing a case where eye swabs were continuously positive for SARS-CoV-2 even after nasopharyngeal swabs turned negative, suggesting the diagnostic value of eye secretions. Therefore, the evidence abstracts uphold the central claim of the news story."
}


Pred



Predicting 1187 :-
evidence list: ['dp4a25me', '2j7omgkd', 'qh23owfb']
{
  "prediction": "True",
  "reason": "The evidence abstracts provided support the claims made in the news paragraph regarding the sensitivity and specificity rates of the NovaLisa SARS-CoV-2 IgG, IgA, and IgM test, as well as the Platelia SARS-CoV-2 Total Ab test. The first and second abstracts confirm the sensitivity and specificity rates for the NovaLisa test as 94.9% and 96.2% for IgG, 89.7% and 98.7% for IgA, and 48.7% and 98.7% for IgM, respectively. They also confirm the Platelia test's specificity and sensitivity as 97.4% and 94.9% for total antibodies. The third abstract does not directly address the NovaLisa or Platelia tests but discusses the iFlash® SARS-CoV-2 assay, which does not contradict the information about the other two tests. Therefore, the news paragraph is factually true based on the evidence provided."
}


Predicting 1188 :-
evidence list: ['diiwq2lj', 'hjdy4hzg', 'd95j1nh8']
{
  "predictio



Predicting 1196 :-
evidence list: ['o1uol5u3', 'o1uol5u3', 'rheq3xes']
{
  "prediction": "True",
  "reason": "The news paragraph claims that SARS-CoV-2 may use β1 integrins as an alternative pathway to infect lung epithelial cells, challenging the prevailing understanding that the virus exclusively uses the ACE2 receptor. The second evidence abstract directly supports this claim by providing experimental evidence that β1 integrins, which are expressed on human pulmonary epithelial cell lines and primary mouse alveolar epithelial cells, can bind to the SARS-CoV-2 spike protein independently of ACE2. This suggests that β1 integrins may function as an alternative receptor for the virus. The first evidence abstract does not contradict the news paragraph but rather focuses on the interaction of the spike protein with ACE2 and TLR4, without excluding the possibility of alternative pathways. Therefore, based on the evidence provided, the news story is factually true."
}


Predicting 1197 :-

In [ ]:
# pip install openai --upgrade

In [35]:
i

1200

In [36]:
len(generated_result)

1200

In [37]:
# messages

In [38]:
# response = get_completion_from_messages(messages)
# print(response)

In [42]:
# result

In [43]:
# generated_result[-1]

In [44]:
# result = dict()
# result['news'] = news
# result['evidence list'] = evidence_list
# result['ground truth'] = item['label']

# try:
#     result.update(json.loads(response))

# except JSONDecodeError as e:
#     pattern = r'"prediction": "(.*?)",\s*"reason": "(.*?)"'
#     match = re.search(pattern, response)
#     if match:
#         result['prediction'] = match.group(1)
#         result['reason'] = match.group(2)
# #             print(dict({'prediction':result['prediction'], 'reason':result['reason']})) 

# generated_result.append(result)

In [45]:
# generated_result[-2:]

In [ ]:
# [i['abstract'] for i in evidence_data if i['cord_uid'] in evidence_list]

In [46]:
os.getcwd()

'C:\\Users\\aishw\\OneDrive\\Desktop\\Research\\Fake Science\\new_task\\new_task'

In [ ]:
# with open('final_output_part2_2.json', 'r') as file:
#     d2 = json.load(file)

In [47]:
with open('final_output_llm_gpt4.json', 'w') as file:
    json.dump(generated_result, file)

In [21]:
# generated_result.remove(generated_result[-1])

In [14]:
with open('final_output_llm_gpt4.json', 'r') as file:
    op = json.load(file)

op[789]

{'news': 'A groundbreaking study has revealed that the 2014 Ebola outbreak in the Democratic Republic of Congo was the result of a clandestine government experiment that went awry, rather than natural human-to-human transmission. The research, published in a reputable scientific journal, indicates that the government was covertly testing a biological weapon named "Ebola-X," engineered to be exceptionally contagious and lethal. This experiment inadvertently triggered the Ebola virus disease (EVD) outbreak, affecting the DRC\'s population. The findings of the study have sparked significant concerns regarding the government\'s role in the outbreak and the actual intentions behind the experiment.',
 'evidence list': ['i8gk3xi1', '1lg2203q', '22ylifqo'],
 'ground truth': 0,
 'prediction': 'False',
 'reason': "None of the provided evidence abstracts mention a clandestine government experiment or a biological weapon named 'Ebola-X.' The first abstract discusses the transmission dynamics of th

In [16]:
for item in summary_data[789:]:
#     print(item)
    
    news = item['abstractive']
    print(news, end='\n')
    
    evidence_list = []
    if type(item['evidence'])==str:
        evidence_list.append(item['evidence'])
    else:
        evidence_list.extend(item['evidence'])
    print(f"evidence list: {evidence_list}")
    print("\n")
    
    evidence = [it['abstract'] for it in evidence_data if it['cord_uid'] in evidence_list]
    evidence = ''.join(evidence)
#     print(evidence_list, end='\n')
    print(evidence)
    
    break

A groundbreaking study has revealed that the 2014 Ebola outbreak in the Democratic Republic of Congo was the result of a clandestine government experiment that went awry, rather than natural human-to-human transmission. The research, published in a reputable scientific journal, indicates that the government was covertly testing a biological weapon named "Ebola-X," engineered to be exceptionally contagious and lethal. This experiment inadvertently triggered the Ebola virus disease (EVD) outbreak, affecting the DRC's population. The findings of the study have sparked significant concerns regarding the government's role in the outbreak and the actual intentions behind the experiment.
evidence list: ['i8gk3xi1', '1lg2203q', '22ylifqo']


The Democratic Republic of Congo (DRC) experienced a confined rural outbreak of Ebola virus disease (EVD) with 69 reported cases from July to October 2014. Understanding the transmission dynamics during the outbreak can provide important information for an